In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd gdrive/My Drive/Colab Notebooks/DerinOgrenme/
%ls

/content/gdrive/My Drive/Colab Notebooks/DerinOgrenme
deneme.jpg      MobileNet61.h5  test/       VGG19_2.h5  VGG19_5.h5
kekik.jpg       MobileNet65.h5  VGG16_1.h5  VGG19_3.h5
MobileNet38.h5  MobileNet73.h5  VGG19_1.h5  VGG19_4.h5


In [3]:
from tensorflow.keras.applications.vgg19  import VGG19
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras import backend as K
from PIL import Image


img_width, img_height = 224, 224

imageChannels = 3
batchSize = 100
epoch = 20
classMode = 'categorical'
fcDense = 3
fcActivation = 'softmax' #son katman
optimizer = 'adam'
loss = 'binary_crossentropy'
metrics = 'accuracy'
train_ornek_sayisi = 3000
validation_ornek_sayisi = 900 


train_data_yolu = 'test/train'
validation_data_yolu = 'test/validation'


if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)



In [4]:
input = Input(input_shape)

model = VGG19(
    include_top = False,
    weights = 'imagenet',
    input_tensor = input,
    classes = 3
)

flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(128, activation='relu', name='fc1')(flat1)
class2 = Dense(256, activation='relu', name='fc2')(class1)
class3= Dense(256, activation='relu', name='fc3')(class2)
class4 = Dense(512, activation='relu', name='fc4')(class3)
class5 = Dense(512, activation='relu', name='fc5')(class4)
class6= Dense(512, activation='relu', name='fc6')(class5)
output = Dense(fcDense, activation='softmax', name='predictions')(class6)

model = Model(inputs=model.inputs, outputs=output)

model.summary()

80150528/80134624 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [5]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [6]:
from tensorflow.keras.applications.vgg19 import preprocess_input


train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2, horizontal_flip=True, zoom_range=0.2, rescale=1./255, rotation_range=45, width_shift_range=0.2,
                                 height_shift_range=0.2, shear_range=0.2, fill_mode='nearest')
test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_data_yolu,
    target_size=(img_width, img_height),
    batch_size=batchSize,
    subset="training",
    class_mode=classMode
)
validation_generator = train_datagen.flow_from_directory(
    train_data_yolu,
    target_size=(img_width, img_height),
    batch_size=batchSize,
    subset="validation",
    class_mode=classMode
)
test_generator = test_datagen.flow_from_directory(
    validation_data_yolu,
    target_size=(img_width, img_height),
    batch_size=batchSize,
    class_mode=classMode
)

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.
Found 900 images belonging to 3 classes.


In [7]:
model.fit(
    train_generator,
    validation_data=validation_generator,
    validation_steps=600//batchSize,
    steps_per_epoch=2400//batchSize,
    batch_size=480,
    epochs=epoch,
    verbose=1
)

Epoch 1/20
24/24 [==============================] - 779s 31s/step - loss: 1.3860 - accuracy: 0.3413 - val_loss: 1.0853 - val_accuracy: 0.3350
Epoch 2/20
24/24 [==============================] - 60s 2s/step - loss: 1.1013 - accuracy: 0.3729 - val_loss: 1.0929 - val_accuracy: 0.3333
Epoch 3/20
24/24 [==============================] - 61s 3s/step - loss: 1.0970 - accuracy: 0.3567 - val_loss: 1.0999 - val_accuracy: 0.3333
Epoch 4/20
24/24 [==============================] - 68s 3s/step - loss: 1.1024 - accuracy: 0.3187 - val_loss: 1.1012 - val_accuracy: 0.3333
Epoch 5/20
24/24 [==============================] - 61s 2s/step - loss: 1.0989 - accuracy: 0.3175 - val_loss: 1.0814 - val_accuracy: 0.3333
Epoch 6/20
24/24 [==============================] - 60s 2s/step - loss: 1.0816 - accuracy: 0.3388 - val_loss: 1.4051 - val_accuracy: 0.3333
Epoch 7/20
24/24 [==============================] - 60s 2s/step - loss: 1.1060 - accuracy: 0.3417 - val_loss: 1.0968 - val_accuracy: 0.3333
Epoch 8/20
24/24 [

In [8]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

img = load_img('kekik.jpg', grayscale=False, color_mode='rgb', target_size=(img_width, img_height))
image = img_to_array(img)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = preprocess_input(image)

yhat = model.predict(image)
print(yhat)

[[0. 0. 1.]]


In [10]:
model.save("VGG19_98.h5")
